# JSON vs orjson – Energy Consumption Analysis

Compares 30 repeated trials of `json.loads` vs `orjson.loads` measured with
EnergiBridge.  Key metrics: **duration**, **CPU energy**, **CORE0 energy**,
and **average CPU power**.

### Column selection

Each CSV contains 42 columns.  Since the focus of this course is
**energy consumption**, we keep only the energy-relevant columns:

| Kept | Reason |
|------|--------|
| `Delta` | Sampling interval (ms) |
| `Time` | Unix timestamp (ms) — used to compute duration |
| `CPU_ENERGY (J)` | Total CPU-package energy counter (RAPL) |
| `CORE0_ENERGY (J)` | Single-core energy counter (RAPL) |

All other columns (16× per-core frequency, 16× per-core usage,
CORE0 frequency / voltage, and all memory / swap columns) are
**dropped** — they are system-state telemetry, not energy measurements.

In [ ]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd()))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.energy_metrics import (
    load_all_runs, remove_outliers, compare_groups, load_run,
)

sns.set_theme(style="whitegrid", context="notebook")
FIGURES = pathlib.Path("figures")
FIGURES.mkdir(exist_ok=True)

## 1. Load and summarise all 30 runs per library

In [ ]:
DATA_ROOT = pathlib.Path("..") 

json_df = load_all_runs(DATA_ROOT / "json_energy")
json_df["library"] = "json"

orjson_df = load_all_runs(DATA_ROOT / "orjson_energy")
orjson_df["library"] = "orjson"

all_df = pd.concat([json_df, orjson_df], ignore_index=True)
print(f"Loaded {len(json_df)} json runs, {len(orjson_df)} orjson runs")
all_df.describe()

## 2. Outlier removal (IQR × 1.5)

Outliers are detected **per library** (json / orjson) using the
**Interquartile Range (IQR)** method on each of the three core metrics:
`duration_s`, `cpu_energy_j`, and `core0_energy_j`.

For each metric the 25th percentile (Q1) and 75th percentile (Q3) are
computed, and the IQR = Q3 − Q1.  Any run whose value falls outside
**[Q1 − 1.5 × IQR,  Q3 + 1.5 × IQR]** is flagged as an outlier.
A run is **removed** if it is an outlier in **any** of the three metrics.

In [ ]:
clean_frames, outlier_frames = [], []
for lib in ("json", "orjson"):
    subset = all_df[all_df["library"] == lib].copy()
    clean, outliers = remove_outliers(subset)
    clean_frames.append(clean)
    outlier_frames.append(outliers)
    n_out = len(outliers)
    print(f"{lib}: kept {len(clean)}/{len(subset)} runs  ({n_out} outlier{'s' if n_out != 1 else ''})")
    if n_out:
        print(f"  removed: {outliers['file'].tolist()}")

clean_df = pd.concat(clean_frames, ignore_index=True)
outlier_df = pd.concat(outlier_frames, ignore_index=True)
clean_df.head()

## 3. Statistical tests (Shapiro → Welch / Mann-Whitney)

Our goal is to answer: **does `orjson` actually consume less energy
(or run faster) than `json`, or are the differences we see in the
box-plots just random noise from run-to-run variation?**  With only
~25 samples per group after outlier removal, eyeballing the plots is
not reliable — we need a p-value to be confident.

**Why Shapiro–Wilk first?**  Energy measurements from repeated runs
are not guaranteed to be normally distributed — background OS activity,
thermal throttling, or swap pressure can skew individual runs.  We run
Shapiro–Wilk on each group to check, because the validity of the next
test depends on it: a parametric test on non-normal data would give
misleading p-values.

**Why Welch's t-test?**  When both groups *are* normal, Welch's t-test
is the strongest option for our setup: two independent groups (json vs
orjson) that may have different variances (e.g. orjson might be more
consistent than json).  It directly tests whether the mean energy /
duration differs between the two libraries.

**Why Mann–Whitney U?**  When Shapiro–Wilk rejects normality for either
group, we cannot trust the t-test.  Mann–Whitney makes no distributional
assumptions, so it remains valid even when our energy data is skewed —
which, as the results below show, happens for most of our metrics.

In [ ]:
METRICS = ["duration_s", "cpu_energy_j", "core0_energy_j", "avg_cpu_power_w"]

json_clean = clean_df[clean_df["library"] == "json"]
orjson_clean = clean_df[clean_df["library"] == "orjson"]

results = {}
for m in METRICS:
    res = compare_groups(json_clean[m].values, orjson_clean[m].values)
    results[m] = res
    sig = "***" if res["p_value"] < 0.001 else "**" if res["p_value"] < 0.01 else "*" if res["p_value"] < 0.05 else "ns"
    print(
        f"{m:20s}  {res['test']:16s}  p={res['p_value']:.4e} {sig}  "
        f"{res['effect_name']}={res['effect_size']:+.3f}  "
        f"json={res['json_mean']:.4f}±{res['json_std']:.4f}  "
        f"orjson={res['orjson_mean']:.4f}±{res['orjson_std']:.4f}"
    )

results_df = pd.DataFrame(results).T
results_df

## 4. Distribution plots (box + strip)

In [ ]:
LABELS = {
    "duration_s": "Duration (s)",
    "cpu_energy_j": "CPU Energy (J)",
    "core0_energy_j": "CORE0 Energy (J)",
    "avg_cpu_power_w": "Avg CPU Power (W)",
}

for metric in METRICS:
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.boxplot(data=clean_df, x="library", y=metric, hue="library", ax=ax, width=0.4,
                palette={"json": "#4C72B0", "orjson": "#DD8452"}, legend=False)
    sns.stripplot(data=clean_df, x="library", y=metric, ax=ax,
                  color="black", alpha=0.35, size=4, jitter=True)
    p = results[metric]["p_value"]
    ax.set_title(f"{LABELS[metric]}  (p = {p:.2e})")
    ax.set_xlabel("")
    ax.set_ylabel(LABELS[metric])
    fig.tight_layout()
    fig.savefig(FIGURES / f"{metric}.png", dpi=150, bbox_inches="tight")
    plt.show()

## 5. Energy vs Duration scatter

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
colors = {"json": "#4C72B0", "orjson": "#DD8452"}
for lib, grp in clean_df.groupby("library"):
    ax.scatter(grp["duration_s"], grp["cpu_energy_j"],
               label=lib, color=colors[lib], s=50, alpha=0.7, edgecolors="white")
ax.set_xlabel("Duration (s)")
ax.set_ylabel("CPU Energy (J)")
ax.set_title("CPU Energy vs Duration per run")
ax.legend(title="Library")
fig.tight_layout()
fig.savefig(FIGURES / "energy_vs_duration.png", dpi=150, bbox_inches="tight")
plt.show()

## 6. Run 30 time-series (CPU Energy and CORE0 Energy over time)

In [ ]:
json_r30 = load_run(DATA_ROOT / "json_energy" / "energy_json_run30.csv")
orjson_r30 = load_run(DATA_ROOT / "orjson_energy" / "energy_orjson_run30.csv")

def normalise(df):
    """Zero-base time and energy columns for easier comparison."""
    d = df.copy()
    d["time_s"] = (d["Time"] - d["Time"].iloc[0]) / 1000.0
    d["cpu_energy_delta"] = d["CPU_ENERGY (J)"] - d["CPU_ENERGY (J)"].iloc[0]
    d["core0_energy_delta"] = d["CORE0_ENERGY (J)"] - d["CORE0_ENERGY (J)"].iloc[0]
    return d

json_r30_n = normalise(json_r30)
orjson_r30_n = normalise(orjson_r30)

TIMESERIES = [
    ("cpu_energy_delta", "CPU Energy (cumulative Δ, J)", "run30_cpu_energy"),
    ("core0_energy_delta", "CORE0 Energy (cumulative Δ, J)", "run30_core0_energy"),
]

for col, title, fname in TIMESERIES:
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(json_r30_n["time_s"], json_r30_n[col], label="json", color="#4C72B0", linewidth=1.2)
    ax.plot(orjson_r30_n["time_s"], orjson_r30_n[col], label="orjson", color="#DD8452", linewidth=1.2)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel(title)
    ax.set_title(f"Run 30 – {title}")
    ax.legend()
    fig.tight_layout()
    fig.savefig(FIGURES / f"{fname}.png", dpi=150, bbox_inches="tight")
    plt.show()